In [97]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [98]:
LINK_LIB_GR = ["torneo=1048", "torneo=1047", "torneo=1046", "torneo=1045", 
                "torneo=1044", "torneo=1043", "torneo=1074", "torneo=1089"]

LINK_LIB_IDA_VUELTA = ["torneo=1048", "torneo=1047", "torneo=1046", "torneo=1045", 
                        "torneo=1044"]

LINK_LIB_UNICA = ["torneo=1043", "torneo=1074", "torneo=1089", "libertadores"]

In [99]:
DICC_MES = {'Enero': '1', 'Febrero': '2', 'Marzo': '3', 'Abril': '4', 'Mayo': '5', 'Junio': '6', 'Julio': '7', 'Agosto': '8', 
            'Septiembre': '9', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'}

DICT_EQUIPOS = {'DepCali':'colombia'}

In [100]:
def revisar_equipo(l, equipo):
    for i in l:
        eq = i.replace(",", "")
        eq = eq.replace(".", "")
        eq = eq.replace(" ", "")
        if eq[:3].upper() == equipo.upper():
            return i
    return "DepCali"

In [101]:
def im_a_equipo(l):
    paises = []
    for i in l:
        im = i.find("img")
        im = im.get("src")
        im = im.split("/")[2].split(".")[0]
        paises.append(im)
    return paises

In [102]:
locales = []
visitas = []
gol_locales = []
gol_visitas = []
ganador = []
goles = []
resultados = []
dif_goles = []
dias = []
anos = []
fases = []

Primero definimos los partidos de grupo

In [103]:
ano = 2014
for num in LINK_LIB_GR:
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_grupos = sopa.find_all('div', class_ = 'grupo')
    for data in lista_grupos:
        equipos = data.find('table', id='posiciones')
        equipos = equipos.find_all('td', align = 'left')
        imagen = im_a_equipo(equipos)
        equipos = [i.text.replace(" ", "") for i in equipos]
        partidos = data.find_all('div', id = 'fixgrupo')

        for i in range(4):
            DICT_EQUIPOS[equipos[i]] = imagen[i]

        for datos in partidos:
            horario = datos.find_all('div', class_ = 'diahoragr')
            res = datos.find_all('div', class_ = 'grtr')

            for j in range(2):
                eq1 = res[j].find('div', class_ = 'greq1')
                eq1 = revisar_equipo(equipos, eq1.text)
                locales.append(eq1)
                eq2 = res[j].find('div', class_ = 'greq2')
                eq2 = revisar_equipo(equipos, eq2.text)
                visitas.append(eq2)

                resultado = res[j].find('div', class_ = 'grres4')
                resultado = resultado.text
                resultado = resultado.split('-')
                res1 = int(resultado[0])
                res2 = int(resultado[1])
                gol_locales.append(res1)
                gol_visitas.append(res2)
                resultados.append(str(res1) + "-" + str(res2))

                if res1 == res2:
                    ganador.append("Empate")
                    dif_goles.append(0)

                elif int(res1) > int(res2):
                    ganador.append("Local")
                    dif_goles.append(int(res1) - int(res2))

                elif int(res1) < int(res2):
                    ganador.append("Visita")
                    dif_goles.append(int(res2) - int(res1))

                goles.append(int(res1) + int(res2))
                anos.append(ano)
                dia = horario[j].text
                dia = dia.replace(",", "") ; dia = dia.split(" ")
                dia = dia[3]
                dias.append(dia)
                fases.append("Grupos")

    ano += 1


Añadimos las libertadores con finales ida y vuelta

In [104]:
LISTA_RONDAS = ["Final", "Final", "Semifinal", "Semifinal", "Cuartos", "Cuartos", "Octavos", "Octavos"]

In [105]:
for num in LINK_LIB_IDA_VUELTA:
    url = f"https://www.promiedos.com.ar/{num}"
    cont = 0
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text ; equ2 = equ[2].text
            locales.append(equ1)
            visitas.append(equ2)

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            resultados.append(res1 + "-" + res2)


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            anos.append(dia[4])
            ano = dia[4]
            dia = dia[1] + "/"  + DICC_MES[dia[3]]
            dias.append(dia)

            fases.append(fase)

        cont += 1

Añadimos las libertadores con final unica

In [106]:
for num in LINK_LIB_UNICA:
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    final_unica = sopa.find_all('div', id='partoff1')
    equ = final_unica[0].find_all('td', class_='game-t1')
    equ = final_unica[0].find_all('td', class_='game-t1')
    equ1 = equ[1].text.replace(' ', '') ; equ2 = equ[2].text.replace(' ', '')
    locales.append(equ1)
    visitas.append(equ2)
    res1 = final_unica[0].find_all('td', class_='game-r1')
    res1 = (res1[1].text)[0]
    gol_locales.append(int(res1))
    res2 = final_unica[0].find_all('td', class_='game-r2')
    res2 = (res2[1].text)[0]
    gol_visitas.append(int(res2))

    if int(res1) == int(res2):
        ganador.append("Empate")
        dif_goles.append(0)

    elif int(res1) > int(res2):
        ganador.append("Local")
        dif_goles.append(int(res1) - int(res2))

    elif int(res1) < int(res2):
        ganador.append("Visita")
        dif_goles.append(int(res2) - int(res1))

    goles.append(int(res1) + int(res2))
    resultados.append(res1 + "-" + res2)


    dia = final_unica[0].find_all('tr', class_='diapart')
    dia = dia[0].text
    dia = dia.split(" ")
    if len(dia) <= 4:
        dia.append("2022")
    anos.append(dia[4])
    ano = dia[4]
    dia = dia[1] + '/' + DICC_MES[dia[3]]
    dias.append(dia)
    fases.append("Final")

In [107]:
for num in LINK_LIB_UNICA:
    url = f"https://www.promiedos.com.ar/{num}"
    cont = 2
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text ; equ2 = equ[2].text
            locales.append(equ1)
            visitas.append(equ2)

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            resultados.append(res1 + "-" + res2)


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            if len(dia) < 5:
                anos.append("2022")
            else:
                anos.append(dia[4])
            dia = dia[1] + "/"  + DICC_MES[dia[3]]
            dias.append(dia)

            fases.append(fase)

        cont += 1
        if cont == 8:
            break

In [108]:
url = "https://www.promiedos.com.ar/libertadores=faseprevia"
page = rq.get(url)
page = bs(page.content, 'html.parser')
tabla = page.find('div', id='tablapts')
tabla = tabla.find_all('div', class_='grupo')
cont = 1
for data in tabla:
    l_equipos = []
    equipo = data.find('table', class_=f'tablesorter{cont}')
    equipo = equipo.find_all('tr')
    equipo.pop(0)
    imgs = im_a_equipo(equipo)
    for nom in equipo:
        nom = nom.find('td', align='left')
        l_equipos.append(nom.text.replace(' ', ''))
    for i in range(4):
        DICT_EQUIPOS[l_equipos[i]] = imgs[i]
    partidos = data.find_all('div', id = 'fixgrupo')
    cont += 1
    for datos in partidos:
        horario = datos.find_all('div', class_ = 'diahoragr')
        res = datos.find_all('div', class_ = 'grtr')

        for j in range(2):
            eq1 = res[j].find('div', class_ = 'greq1')
            eq1 = revisar_equipo(l_equipos, eq1.text)
            locales.append(eq1)
            eq2 = res[j].find('div', class_ = 'greq2')
            eq2 = revisar_equipo(l_equipos, eq2.text)
            visitas.append(eq2)

            resultado = res[j].find('div', class_ = 'grres4')
            resultado = resultado.text
            resultado = resultado.split('-')
            res1 = int(resultado[0])
            res2 = int(resultado[1])
            gol_locales.append(res1)
            gol_visitas.append(res2)
            resultados.append(str(res1) + "-" + str(res2))

            if res1 == res2:
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            anos.append(ano)
            dia = horario[j].text
            dia = dia.replace(",", "") ; dia = dia.split(" ")
            dia = dia[3]
            dias.append(dia)
            fases.append("Grupos")


In [109]:
pais_locales = []
for i in locales:
    i = i.replace(" ", "")
    pais_locales.append(DICT_EQUIPOS[i])

pais_visitas = []
for i in visitas:
    i = i.replace(" ", "")
    pais_visitas.append(DICT_EQUIPOS[i])

In [110]:
df_libertadores = pd.DataFrame()
df_libertadores["Local"] = locales
df_libertadores["Pais_local"] = pais_locales
df_libertadores["Visita"] = visitas
df_libertadores["Pais_visita"] = pais_visitas
df_libertadores["Gol_local"] = gol_locales
df_libertadores["Gol_visita"] = gol_visitas
df_libertadores["Ganador"] = ganador
df_libertadores["Resultado"] = resultados
df_libertadores["Año"] = anos
df_libertadores["Fase"] = fases
df_libertadores["Goles"] = goles
df_libertadores["Diferencia_goles"] = dif_goles
df_libertadores["Dia"] = dias
df_libertadores

,Local,Pais_local,Visita,Pais_visita,Gol_local,Gol_visita,Ganador,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
0,Universitario(P),peru,Velez,argentina,0,1,Visita,0-1,2014,Grupos,1,1,11/2
1,Paranaense,brasil,TheStrongest,bolivia,1,0,Local,1-0,2014,Grupos,1,1,13/2
2,TheStrongest,bolivia,Universitario(P),peru,1,0,Local,1-0,2014,Grupos,1,1,20/2
3,Velez,argentina,Paranaense,brasil,2,0,Local,2-0,2014,Grupos,2,2,25/2
4,TheStrongest,bolivia,Velez,argentina,2,0,Local,2-0,2014,Grupos,2,2,11/3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,Sp.Cristal,peru,U.Catolica,chile,1,1,Empate,1-1,2022,Grupos,2,0,4/5
1112,Flamengo,brasil,U.Catolica,chile,3,0,Local,3-0,2022,Grupos,3,3,17/5
1113,Sp.Cristal,peru,Talleres(C),argentina,0,0,Empate,0-0,2022,Grupos,0,0,17/5
1114,Flamengo,brasil,Sp.Cristal,peru,2,1,Local,2-1,2022,Grupos,3,1,24/5
